In [3]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.eager as tfe

print(tf.__version__)
print(tf.__git_version__)

tfe.enable_eager_execution()

Instructions for updating:
Use the retry module or similar alternatives.
1.7.0
v1.7.0-3-g024aecf414


In [4]:
# set data dimensions
K = 3
V = 5
D = 100
N = 100

# set the seed
np.random.seed(2014)

# beta prior parameters
eta = np.ones(V) * 1e-1

# beta profiles
beta = np.random.dirichlet(alpha=eta, size=K)

# theta prior parameters
alpha = np.ones(K) * 1e-1
# alpha[0] = 10

# document's prior topic allocation
theta = np.random.dirichlet(alpha=alpha, size=D)

# word's topic membership
z = [np.random.choice(K, size=N, replace=True, p=theta[d, :]) for d in range(D)]
z = np.vstack(z)

# actual words and counts
w = [np.array([np.random.choice(V, size=1, p=beta[k,:])[0] for k in z[d, :]]  + list(range(V))) for d in range(D)]
nw = [np.unique(w[d], return_counts=True)[1] for d in range(D)]
nw = np.vstack(nw)
w = np.vstack(w)

nw = tf.convert_to_tensor(nw, dtype=tf.float32)
nw = tfe.Variable(initial_value=tf.transpose(nw),
                 name="nw_vd")
# nw

In [5]:
nw

<tf.Variable 'nw_vd:0' shape=(5, 100) dtype=float32, numpy=
array([[100.,  95.,  91.,  18.,  77.,  72.,  91.,  33.,  92.,  67.,   4.,
        101.,  93.,  94., 101.,  87.,   1.,  59., 101.,  98.,  98.,   2.,
         89.,  99.,  94.,  99.,  84., 100.,  97.,  39.,   1.,  99.,  99.,
         97.,  95.,  90.,  96.,  90., 101.,  61.,   6.,  99.,  92., 100.,
         99.,  98.,   1.,  97.,  80., 100.,  95.,  98.,  46.,   1.,  59.,
         96.,  97.,  74.,  64.,   6.,  97.,  26.,  87.,  92.,  98.,  98.,
         93.,  69.,  70.,  99.,  81.,  93.,  56.,  92.,  95., 101.,  96.,
         95.,  51., 101.,   6.,  62.,  92.,   1.,  97.,  12.,  91.,   1.,
         95.,  20., 100.,   2.,  92.,   4.,  91.,  78., 100.,  77.,  95.,
         93.],
       [  1.,   2.,   2.,   1.,   1.,   3.,   5.,   2.,   1.,   2.,   1.,
          1.,   3.,   3.,   1.,   2.,   1.,   1.,   1.,   1.,   1.,   1.,
          1.,   1.,   3.,   1.,   4.,   1.,   2.,   1.,   1.,   1.,   1.,
          3.,   2.,   3.,   2.,   1.,

In [6]:
# initialize LDA parameters
def initialize_variables(K, V, D, alpha=1e-1, eta=1e-1, seed=2014):
    """
    Initialize parameters of LDA model returning adequate Tensors.

    args:
    
        K (int): number of LDA components 
        V (int): vocabulary size
        D (int): number of documents
        alpha (float): hyperparameter for theta prior
        eta (float): hyperparameter for beta prior
       
       
    returns:
    
        eta: [V] tensor with prior parameters (alpha) for beta
        lambda: [K, V] tensor with posterior word distribution per class
        phi: [K, V, D] tensor with vocabulary membership per document
        gamma: [K, D] tensor
        
    """
    tf.set_random_seed(seed)
    eta = tfe.Variable(initial_value=tf.ones(V) * eta, 
                       name="eta_v")
    alpha = tfe.Variable(initial_value=tf.ones(K) * alpha, 
                         name="alpha_k")    
    lam = tfe.Variable(initial_value=tf.abs(tf.random_normal(shape=(K, V))), 
                       name="lambda_kv")
    
    phi = tfe.Variable(initial_value=tf.random_normal(shape=(K, V, D)), 
                       name="phi_kvd")
    tf.assign(ref=phi, value=tf.nn.softmax(phi, axis=0))
    
    gamma = tfe.Variable(initial_value=tf.abs(tf.random_normal(shape=(K, D))), 
                        name="gamma_kd")
    
    e_log_beta = tfe.Variable(initial_value=tf.abs(tf.random_normal(shape=(K, V, D))) * .0, 
                        name="e_log_beta_kvd")
    
    e_log_theta = tfe.Variable(initial_value=tf.abs(tf.random_normal(shape=(K, V, D))) * .0, 
                        name="e_log_theta_kvd")
    
    return eta, alpha, lam, phi, gamma, e_log_beta, e_log_theta

# test
eta, alpha, lam, phi, gamma, e_log_beta, e_log_theta = initialize_variables(K, V, D)



In [7]:
# lambda update
def update_lambda(lam, eta, phi, nw):
    
    K = lam.shape.as_list()[0]
    for k in range(K):
        tf.scatter_update(ref=lam, 
                  indices=k, 
                  updates=tf.reduce_sum(tf.multiply(phi[k], nw), axis=1) + eta)
        
    return lam

# test
# update_lambda(lam, eta, phi, nw)
# print(lam)

In [8]:
# gamma update
def update_gamma(gamma, alpha, phi, nw):
    
    K = gamma.shape.as_list()[0]
    for k in range(K):
        tf.scatter_update(ref=gamma, 
                  indices=k, 
                  updates=tf.reduce_sum(tf.multiply(phi[k], nw), axis=0) + alpha[k])

        
    return gamma

tmp = gamma.value()
update_gamma(gamma, alpha, phi, nw)
print(gamma)


<tf.Variable 'gamma_kd:0' shape=(3, 100) dtype=float32, numpy=
array([[33.179447 , 28.432518 , 12.622646 , 17.165543 , 15.577526 ,
        10.490307 , 11.648556 , 32.405262 , 35.44801  , 46.716198 ,
         9.907303 , 71.49604  , 45.32922  , 27.23914  , 67.28874  ,
        49.506325 , 80.40026  , 33.695103 , 95.63079  , 70.015205 ,
        16.508184 , 20.57544  , 20.053911 , 26.896889 , 32.449123 ,
        16.200172 , 34.788353 , 68.103065 ,  9.126976 , 22.359228 ,
         4.256766 , 18.665571 , 22.737452 , 61.869286 , 42.570847 ,
         9.795437 , 14.991373 , 52.058765 , 35.303062 , 33.66349  ,
        35.514534 , 62.23464  , 13.184516 , 84.8964   , 87.078835 ,
        39.539482 ,  9.8654995, 59.58053  , 47.186913 ,  8.718346 ,
        62.436123 , 50.3837   , 40.483505 , 25.55224  , 34.0292   ,
        29.543427 ,  6.3052363, 44.445526 , 12.977353 , 18.668716 ,
        77.82936  , 61.49327  , 38.4598   , 68.482994 , 45.963047 ,
        95.63182  , 55.1969   , 37.85514  , 15.071856

In [9]:
tmp

<tf.Tensor: id=132, shape=(3, 100), dtype=float32, numpy=
array([[0.856814  , 0.17276734, 2.0289466 , 0.36276183, 1.5049754 ,
        0.01352629, 0.8085556 , 0.05183626, 0.1407165 , 0.2526934 ,
        1.7273577 , 0.15571633, 0.38100538, 0.26664335, 0.9372891 ,
        0.5413353 , 1.3022376 , 0.1799982 , 1.3224765 , 0.13220988,
        0.46405008, 1.3386966 , 1.1340383 , 0.9544467 , 1.0398598 ,
        0.45572257, 0.41327694, 0.42305326, 1.7513535 , 1.2226332 ,
        1.3665848 , 0.69009304, 0.19026844, 0.9641658 , 1.1776    ,
        1.1071438 , 0.13112977, 0.8296427 , 0.79165363, 0.07314051,
        1.2809435 , 1.5483049 , 0.44105747, 0.32301107, 0.6834025 ,
        1.708542  , 0.08623107, 0.2677692 , 1.0648154 , 0.91856754,
        1.791696  , 0.58421534, 0.98128223, 0.99504936, 1.5844259 ,
        0.58608973, 0.20972344, 0.66854095, 0.9684806 , 0.9942798 ,
        0.34617952, 1.2616568 , 0.36821425, 0.5178254 , 0.6032014 ,
        0.25295106, 0.5769268 , 0.5076303 , 3.0258594 , 0.

In [10]:
def update_e_log_beta(e_log_beta, lam):
    
    K = lam.shape.as_list()[0]
    for k in range(K):
        tf.scatter_update(ref=e_log_beta,
                  indices=k,
                  updates=tf.tile(tf.expand_dims(tf.digamma(lam[k]) - tf.digamma(tf.reduce_sum(lam[k])), axis=1), multiples=[1, D]))
    
    return e_log_beta

print(e_log_beta)
update_e_log_beta(e_log_beta, lam);
print(e_log_beta)

<tf.Variable 'e_log_beta_kvd:0' shape=(3, 5, 100) dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)>
<tf.Variable 'e_log_beta_kvd:0' shape=(3, 5, 100) dtype=float32, numpy=
array([[[ -4.3613567,  -4.3613567,  -4.3613567, ...,  -4.3613567,
          -4.3613567,  -4.3613567],
        [ -1.004497 ,  -1.004497 ,  -1.004497 , ...,  -1.004497 ,
          -1.004497 ,  -1.004497 ],
        [ -3.9851837,  -3.9851837,  -3.985183

In [11]:
def update_e_log_theta(e_log_theta, gamma):
    
    tf.assign(ref=e_log_theta, 
              value=tf.tile(tf.expand_dims(tf.digamma(gamma) - tf.digamma(tf.reduce_sum(gamma, axis=0)), axis=1), multiples=[1, V, 1]))

    return e_log_theta

update_e_log_theta(e_log_theta, gamma)

<tf.Variable 'e_log_theta_kvd:0' shape=(3, 5, 100) dtype=float32, numpy=
array([[[-1.165272  , -1.3222127 , -2.1566992 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.165272  , -1.3222127 , -2.1566992 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.165272  , -1.3222127 , -2.1566992 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.165272  , -1.3222127 , -2.1566992 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.165272  , -1.3222127 , -2.1566992 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ]],

       [[-2.0262158 , -0.52194834, -0.5760498 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
        [-2.0262158 , -0.52194834, -0.5760498 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
        [-2.0262158 , -0.52194834, -0.5760498 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
        [-2.0262158 , -0.52194834, -0.5760498 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
       

In [12]:
e_log_theta

<tf.Variable 'e_log_theta_kvd:0' shape=(3, 5, 100) dtype=float32, numpy=
array([[[-1.165272  , -1.3222127 , -2.1566992 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.165272  , -1.3222127 , -2.1566992 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.165272  , -1.3222127 , -2.1566992 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.165272  , -1.3222127 , -2.1566992 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ],
        [-1.165272  , -1.3222127 , -2.1566992 , ..., -0.71888995,
         -2.0216992 , -2.9808097 ]],

       [[-2.0262158 , -0.52194834, -0.5760498 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
        [-2.0262158 , -0.52194834, -0.5760498 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
        [-2.0262158 , -0.52194834, -0.5760498 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
        [-2.0262158 , -0.52194834, -0.5760498 , ..., -1.0551589 ,
         -0.36708975, -1.6438904 ],
       

In [13]:
e_log_beta

<tf.Variable 'e_log_beta_kvd:0' shape=(3, 5, 100) dtype=float32, numpy=
array([[[ -4.3613567,  -4.3613567,  -4.3613567, ...,  -4.3613567,
          -4.3613567,  -4.3613567],
        [ -1.004497 ,  -1.004497 ,  -1.004497 , ...,  -1.004497 ,
          -1.004497 ,  -1.004497 ],
        [ -3.9851837,  -3.9851837,  -3.9851837, ...,  -3.9851837,
          -3.9851837,  -3.9851837],
        [ -3.3059347,  -3.3059347,  -3.3059347, ...,  -3.3059347,
          -3.3059347,  -3.3059347],
        [ -2.936366 ,  -2.936366 ,  -2.936366 , ...,  -2.936366 ,
          -2.936366 ,  -2.936366 ]],

       [[ -2.9444265,  -2.9444265,  -2.9444265, ...,  -2.9444265,
          -2.9444265,  -2.9444265],
        [ -3.2798169,  -3.2798169,  -3.2798169, ...,  -3.2798169,
          -3.2798169,  -3.2798169],
        [ -1.8179834,  -1.8179834,  -1.8179834, ...,  -1.8179834,
          -1.8179834,  -1.8179834],
        [ -0.9550425,  -0.9550425,  -0.9550425, ...,  -0.9550425,
          -0.9550425,  -0.9550425],
        

In [14]:
import time
start = time.time()

def update_phi(e_log_beta, e_log_theta):
    tf.assign(ref=phi, 
              value=e_log_beta + e_log_theta)
    tf.assign(ref=phi, value=tf.nn.softmax(logits=phi, axis=0))
    return phi


update_phi(e_log_beta, e_log_theta)

end = time.time()
print(end - start)
print(phi)

0.0018720626831054688
<tf.Variable 'phi_kvd:0' shape=(3, 5, 100) dtype=float32, numpy=
array([[[3.6447594e-01, 9.8216988e-02, 4.7535326e-02, ...,
         2.5338078e-01, 4.4296607e-02, 5.9869461e-02],
        [4.6129033e-01, 6.4811915e-01, 3.0940506e-01, ...,
         7.8280282e-01, 4.2770311e-01, 9.2391737e-02],
        [2.1311277e-01, 4.8917525e-02, 2.3025669e-02, ...,
         1.3812724e-01, 2.1419320e-02, 2.9195052e-02],
        [5.5261429e-02, 3.5842240e-02, 1.3987195e-02, ...,
         9.2634402e-02, 1.5405496e-02, 6.7957784e-03],
        [2.2242963e-01, 4.3187967e-01, 1.4157836e-01, ...,
         5.7575411e-01, 2.3359051e-01, 3.2930043e-02]],

       [[6.3552213e-01, 9.0178287e-01, 9.5246434e-01, ...,
         7.4661893e-01, 9.5570326e-01, 9.4012803e-01],
        [2.0040676e-02, 1.4826766e-01, 1.5446705e-01, ...,
         5.7471670e-02, 2.2991712e-01, 3.6148567e-02],
        [7.8688163e-01, 9.5108211e-01, 9.7697335e-01, ...,
         8.6187208e-01, 9.7858036e-01, 9.7079843e-01],

In [15]:
nw_kvd = tf.tile(tf.expand_dims(nw / tf.reduce_sum(nw), axis=0), 
                 multiples=[K, 1, 1])
nw_kvd

<tf.Tensor: id=292, shape=(3, 5, 100), dtype=float32, numpy=
array([[[9.5238099e-03, 9.0476191e-03, 8.6666662e-03, ...,
         7.3333331e-03, 9.0476191e-03, 8.8571431e-03],
        [9.5238094e-05, 1.9047619e-04, 1.9047619e-04, ...,
         9.5238094e-05, 9.5238094e-05, 5.7142857e-04],
        [1.9047619e-04, 9.5238094e-05, 9.5238094e-05, ...,
         1.9047619e-04, 9.5238094e-05, 9.5238094e-05],
        [9.5238094e-05, 1.9047619e-04, 9.5238094e-05, ...,
         2.2857143e-03, 9.5238094e-05, 9.5238094e-05],
        [9.5238094e-05, 4.7619047e-04, 9.5238094e-04, ...,
         9.5238094e-05, 6.6666666e-04, 3.8095238e-04]],

       [[9.5238099e-03, 9.0476191e-03, 8.6666662e-03, ...,
         7.3333331e-03, 9.0476191e-03, 8.8571431e-03],
        [9.5238094e-05, 1.9047619e-04, 1.9047619e-04, ...,
         9.5238094e-05, 9.5238094e-05, 5.7142857e-04],
        [1.9047619e-04, 9.5238094e-05, 9.5238094e-05, ...,
         1.9047619e-04, 9.5238094e-05, 9.5238094e-05],
        [9.5238094e-05, 1

In [16]:
def elbo(phi, e_log_beta, e_log_theta, nw_kvd):

    A = tf.reduce_sum(nw_kvd * phi * (e_log_beta + e_log_theta - tf.log(phi + 1e-6)))
    
    
    return A.numpy()

elbo(phi, e_log_beta, e_log_theta, nw_kvd)    

-3.250828

In [17]:
seed = 1

In [18]:
seed += 1
eta, alpha, lam, phi, gamma, e_log_beta, e_log_theta = initialize_variables(K, V, D)

prev_elbo = 0.0
next_elbo = 0.0
iter = 0

for i in range(100000):
    
    for j in range(100000):
        # E-Step:
        update_e_log_beta(e_log_beta, lam);
        update_e_log_theta(e_log_theta, gamma);
        update_phi(e_log_theta=e_log_theta, e_log_beta=e_log_beta)
        gamma_prev = gamma.value()
        update_gamma(gamma, alpha, phi, nw)
        
        diff = tf.reduce_mean(tf.abs(gamma_prev - gamma.value()))
        if diff < 1e-6:
            break
    
    # M-Step:
    update_lambda(lam, eta, phi, nw)
    
    
    next_elbo = elbo(phi, e_log_beta, e_log_theta, nw_kvd)
#     next_elbo = 0.0
    print("Iteration:", iter, "ELBO:", next_elbo)
    
    diff = np.abs(next_elbo - prev_elbo)
    if diff < 1e-6:
        print("Converged!")
        break
    else:
        iter += 1
        prev_elbo = next_elbo



Iteration: 0 ELBO: -2.693914
Iteration: 1 ELBO: -0.7925892
Iteration: 2 ELBO: -0.7924225
Iteration: 3 ELBO: -0.79242504
Iteration: 4 ELBO: -0.79242784
Iteration: 5 ELBO: -0.7924291
Iteration: 6 ELBO: -0.7924297
Converged!


In [19]:
print(np.round((beta), decimals=3))
print(np.transpose(np.round(tf.transpose(lam) / tf.reduce_sum(lam, axis=1), decimals=3)))

[[0.    0.001 0.005 0.992 0.003]
 [0.99  0.001 0.009 0.    0.   ]
 [0.926 0.016 0.001 0.004 0.053]]
[[0.901 0.018 0.013 0.037 0.031]
 [0.695 0.016 0.014 0.242 0.032]
 [0.001 0.008 0.001 0.984 0.006]]


In [21]:
import pandas as pd

In [22]:
data = pd.read_csv("NIPS_1987-2015.csv")

In [23]:
nw = np.array(data.iloc[:, 1:])
nw = nw.astype('float32')
nw = nw.transpose()

In [24]:
nw = tf.convert_to_tensor(nw)
# nw = nw[0:1000, :]
nw = tf.transpose(nw)

In [25]:
K = 10
V, D = nw.shape

In [26]:
nw_kvd = tf.tile(tf.expand_dims(nw / tf.reduce_sum(nw), axis=0), 
                 multiples=[K, 1, 1])

In [27]:
seed = 1
eta, alpha, lam, phi, gamma, e_log_beta, e_log_theta = initialize_variables(K, V, D, alpha=1e-3, eta=1e-3)

In [31]:
elbo(phi, e_log_beta, e_log_theta, nw_kvd)

1.9235986

In [ ]:
max_iter = 10
max_iter_e_step = 10

prev_elbo = 0.0
next_elbo = 0.0
iter = 0

import time
start = time.time()

for i in range(max_iter):
    
    for j in range(max_iter_e_step):
        # E-Step:
        update_e_log_beta(e_log_beta, lam);
        update_e_log_theta(e_log_theta, gamma);
        update_phi(e_log_theta=e_log_theta, e_log_beta=e_log_beta)
        gamma_prev = gamma.value()
        update_gamma(gamma, alpha, phi, nw)
        diff = tf.reduce_mean(tf.abs(gamma_prev - gamma.value()))
        if diff < 1e-3:
            break
    
    # M-Step:
    update_lambda(lam, eta, phi, nw)
    
    
    next_elbo = elbo(phi, e_log_beta, e_log_theta, nw_kvd)
    print("Iteration:", iter, "ELBO:", next_elbo)
    
    
    diff = np.abs(next_elbo - prev_elbo)
    if diff < 1e-6:
        print("Converged!")
        break
    else:
        iter += 1
        prev_elbo = next_elbo

        
end = time.time()
print(end - start)


In [37]:
print((end - start) / 3600)

2.572712027364307


In [41]:
print("Iteration:", iter, "ELBO:", next_elbo)
diff

Iteration: 10 ELBO: -7.7598295


0.009083748

<tf.Tensor: id=47591, shape=(10, 5811), dtype=float32, numpy=
array([[3.10704231e-01, 4.31428969e-01, 2.57008106e-01, ...,
        4.67069412e-07, 4.78466461e-07, 4.87802595e-07],
       [4.87739407e-02, 2.10148886e-01, 2.10566476e-01, ...,
        4.67069412e-07, 1.13660265e-02, 7.59115145e-02],
       [2.11768657e-01, 4.61465305e-07, 1.36448264e-01, ...,
        4.52919096e-01, 6.49587631e-01, 4.87802595e-07],
       ...,
       [1.11565702e-01, 1.27184093e-01, 1.36335418e-01, ...,
        4.67069412e-07, 4.78466461e-07, 8.18663687e-02],
       [1.17566949e-02, 4.61465305e-07, 8.53963968e-07, ...,
        1.91637091e-02, 3.43658105e-02, 2.96117544e-01],
       [1.05418310e-01, 5.26086725e-02, 9.31278169e-02, ...,
        1.81051955e-01, 1.02145359e-01, 3.26492637e-01]], dtype=float32)>

In [94]:
# topic term distribution:
topic_term_dist = (tf.transpose(tf.transpose(lam) / tf.reduce_sum(lam, axis=1))).numpy()
print(topic_term_dist.shape)

# doc_topic_dists :array-like, shape (n_docs, n_topics)
doc_topic_dist = tf.transpose(gamma / tf.reduce_sum(gamma, axis=0)).numpy()
print(doc_topic_dist.shape)

# # doc_lengths :array-like, shape n_docs
doc_len = tf.reduce_sum(nw, axis=0)
doc_len = doc_len.numpy()
print(doc_len.shape)

# # vocab :array-like, shape n_terms
vocab = data.iloc[:, 0].values
print(vocab.shape)

# # term_frequency :array-like, shape n_terms
term_frec = tf.reduce_sum(nw, axis=1)
term_frec = term_frec.numpy()
print(term_frec.shape)

(10, 11463)
(5811, 10)
(5811,)
(11463,)
(11463,)


In [95]:
import pyLDAvis
topics = pyLDAvis.prepare(topic_term_dist, doc_topic_dist, doc_len, vocab, term_frec)
pyLDAvis.save_html(topics, fileobj="results2.html")

/home/ubuntu/.local/share/virtualenvs/tf-lda-Y7BvcYp3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [70]:
lam

<tf.Variable 'lambda_kv:0' shape=(10, 11463) dtype=float32, numpy=
array([[1.00000005e-03, 1.00706820e-03, 1.69346985e+02, ...,
        1.00000005e-03, 5.43059254e+00, 1.00000005e-03],
       [4.18568649e+01, 1.00000005e-03, 1.53377876e+01, ...,
        1.56098795e+01, 5.63311462e+01, 4.30760193e+01],
       [1.00000005e-03, 1.00000005e-03, 1.00000005e-03, ...,
        1.49566793e+01, 2.13392884e-01, 1.00000005e-03],
       ...,
       [1.00000005e-03, 1.00000005e-03, 1.63188601e+00, ...,
        3.66662526e+00, 9.18198490e+00, 1.00000005e-03],
       [4.25054169e+01, 1.00000005e-03, 1.00000005e-03, ...,
        1.00000005e-03, 1.18314829e+01, 1.53055573e+00],
       [1.24424562e+01, 1.60802937e+01, 1.00000005e-03, ...,
        1.00000005e-03, 1.23185318e+02, 7.23964157e+01]], dtype=float32)>